In [35]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from sympy import pretty_print
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
import os

### load files

In [36]:
loader = DirectoryLoader("./files_used")
docs = loader.load()


### Embedding

In [37]:
# Embeddings and vector store
import shutil
from langchain.vectorstores import Chroma


# from langchain_community.embeddings.sentence_transformer import (
#     SentenceTransformerEmbeddings,
# )

# persist_directory = './db/chromas-e5-mistral'
# persist_directory = './db/chromas-camembert'
persist_directory = './db/chromas-e5-large-semantic'

In [38]:
import os 

if os.path.exists("model_files/e5-large/multilingual-e5-large"):
    print("Model already exists")
else:
    print("Downloading model")

Model already exists


In [39]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
# modelPath = "model_files/dangvantuan/sentence-camembert-large"
# modelPath = "model_files/Salesforce/SFR-Embedding-Mistral"
# modelPath = "model_files/mistral-e5/e5-mistral-7b-instruct"
modelPath = "model_files/e5-large/multilingual-e5-large"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [49]:
!pip install -qU langchain_experimental langchain ragas chromadb langchain-groq fastembed 

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-inference 0.6.1 requires onnx==1.14.1, but you have onnx 1.16.1 which is incompatible.


In [ ]:
!pip install 'onnx==1.14.1' --force-reinstall

In [47]:
def all_splitter(docs):
    # Markdown 
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
        md_docs.extend(md_doc)
    

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400, chunk_overlap=200, add_start_index=True, separators=['\n\n', '\n', '(?<=\. )']
    )
    all_splits = text_splitter.split_documents(md_docs)
    return all_splits

def markdown_splitter(docs):
    # Markdown 
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
        md_docs.extend(md_doc)
    
    return md_docs

def semantic_splitter(docs):
    # Vérifier si la liste de documents est vide
    if not docs:
        raise ValueError("The input document list is empty.")
    
    # Vérifier si les contenus des pages des documents sont vides
    for d in docs:
        if not d.page_content:
            raise ValueError(f"Document {d} has empty page content.")

    # Split based on semantic similarity
    semantic_chunker = SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile"
    )
    
    # Extraire le contenu des pages
    page_contents = [d.page_content for d in docs]

    # S'assurer que le contenu des pages n'est pas vide
    if not page_contents:
        raise ValueError("The extracted page content list is empty.")
    
    try:
        semantic_chunks = semantic_chunker.create_documents(page_contents)
    except Exception as e:
        print(f"An error occurred while creating semantic chunks: {e}")
        raise
    return semantic_chunks


In [46]:
for d in docs:
    print(d.metadata)
    print(d.page_content)
    print("")

{'source': 'files_used\\aidant.txt'}
# Aidant

Gan Assurances s'engage pour soutenir les salariés aidants . Aujourd’hui, on compte 11 millions d’aidants. D’ici 2030, 1 actif sur 4 aura, en plus de son emploi, à s’occuper d’un proche. L'«aidant» est défini par la loi depuis 2015 comme une : « personne qui vient en aide, de manière régulière et fréquente, à titre non professionnel, pour accomplir tout ou partie des actes ou des activités de la vie quotidienne d’une personne en perte d’autonomie, du fait de l’âge, de la maladie ou d’un handicap. » Conscient de ces enjeux depuis plusieurs années, Gan Assurances met en place des mesures destinées à favoriser la conciliation de la vie professionnelle et de la vie familiale de ces salariés aidants. Ces dispositifs résultent de l’accord UES relatif aux salariés en situation de handicap ou accompagnant un membre de leur famille touché par la maladie ou le handicap du 20 novembre 2020. Est également concerné, l’Accord relatif à la Diversité et à

In [48]:
chunks = semantic_splitter(docs)

TypeError: __init__() got an unexpected keyword argument 'breakpoint_threshold_type'

In [ ]:
# If the directory exists, first delete it
try:
    shutil.rmtree(persist_directory)
except FileNotFoundError as e:
    pass

# Create vector store and save the db
db = Chroma.from_documents(
    chunks, 
    embeddings,
    persist_directory=persist_directory
)
db.persist()

In [ ]:
db._collection.count()

### Chroma connection

In [3]:
# Embeddings and vector store
from langchain.vectorstores import Chroma

persist_directory = './db/chromas-e5-large'

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
# modelPath = "model_files/dangvantuan/sentence-camembert-large"
# modelPath = "model_files/Salesforce/SFR-Embedding-Mistral"
# modelPath = "model_files/mistral-e5/e5-mistral-7b-instruct"
modelPath = "model_files/e5-large/multilingual-e5-large"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [5]:
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [14]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
)


### LLM Model

In [7]:
import torch

torch.cuda.is_available()

True

In [8]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline


MODEL_NAME = "model_files/mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True,local_files_only=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map="auto",
    local_files_only=True,
    # quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:


generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    max_new_tokens=1048,
)

In [10]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

### Chain

In [17]:
from langchain_core.prompts import PromptTemplate

template = """Instruction: Vous êtes un assistant des ressources humaine au sein de la compagnie d'assurance de Gan Assurances. Des questions et des données de contexte vous seront fournies. Répondez aux questions en utilisant les données de contexte fournies. Si la réponse n'est pas dans le contexte ou que le contexte ne correspond pas à la question, répondez simplement par "Je ne sais pas".
De plus, vous répondrez en français.

{context}

Question: {question}

Réponse:"""


prompt = PromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

> ### Classic ask

In [48]:


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [49]:
rag_chain.invoke("Combien ais-je de jours de congés par an ?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" Selon votre statut, vous disposez de 205 jours travaillés par an. Cependant, vous pouvez prendre jusqu'à 20 jours de congés payés par an. En outre, il existe des congés exceptionnels comme ceux liés à l'obtention d'un permis de conduire ou à un déménagement."

> ### Ask with source documents

In [50]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [51]:
response = rag_chain_with_source.invoke("Combien ais-je de jours de congés par an ?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [52]:
response["context"]

[Document(page_content="Vous pouvez bénéficier de jours d'absences autorisées à caractère exceptionnel, pour les motifs ci-dessous. Permis de conduire \u200b\u200b\u200b\u200b\u200b\u200b\u200bIl est possible de prendre 1/2 journée de congé en vue de l'examen pour l'obtention du permis de conduire. Déménagement Les salariés titulaires peuvent prendre 1 journée de congé au titre d'un déménagement (une fois par an).", metadata={'Sous-titre 1': 'Autres congés :', 'Titre 1': 'Congés enfants', 'source': 'files_used\\enfants.txt'}),
 Document(page_content='La durée des congés payés pouvant être pris en une seule fois ne peut excéder 20 jours ouvrés. Le congé principal (congés d’été) doit être pris du 1er mai au 31 octobre pour une durée minimum de 10 jours ouvrés consécutifs et ne dépassant pas 20 jours ouvrés. Il est cependant recommandé que les congés d’été soient pris, dans la mesure du possible, entre le 1er juin et le 30 septembre. Les autres jours peuvent être pris en dehors de la péri

In [53]:

cont = ""
for context in response["context"]:
        cont += context.page_content + "\n\n"
cont

"Vous pouvez bénéficier de jours d'absences autorisées à caractère exceptionnel, pour les motifs ci-dessous. Permis de conduire \u200b\u200b\u200b\u200b\u200b\u200b\u200bIl est possible de prendre 1/2 journée de congé en vue de l'examen pour l'obtention du permis de conduire. Déménagement Les salariés titulaires peuvent prendre 1 journée de congé au titre d'un déménagement (une fois par an).\n\nLa durée des congés payés pouvant être pris en une seule fois ne peut excéder 20 jours ouvrés. Le congé principal (congés d’été) doit être pris du 1er mai au 31 octobre pour une durée minimum de 10 jours ouvrés consécutifs et ne dépassant pas 20 jours ouvrés. Il est cependant recommandé que les congés d’été soient pris, dans la mesure du possible, entre le 1er juin et le 30 septembre. Les autres jours peuvent être pris en dehors de la période de mai à octobre, de préférence par semaine entière afin de permettre une meilleure organisation de l’activité. En tout état de cause les congés payés doiv

> ### Passing manual docs + Q&A

In [1]:
question = "combien de jours de congés ?"

In [15]:
docs = retriever.get_relevant_documents(question, consider_metadata=True)

In [32]:
docs[3].metadata["source"]

'files_used\\enfants.txt'

In [18]:
from langchain_core.runnables import RunnablePassthrough, RunnablePick

# Chain
chain = (
    RunnablePassthrough.assign(context=RunnablePick("context") | format_docs)
    | prompt
    | llm
    | StrOutputParser()
)

# Run
chain.invoke({"context": docs, "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" Selon votre statut, vous pouvez avoir jusqu'à 205 jours travaillés par an. Les congés payés ne peuvent excéder 20 jours ouvrés (lundi à vendredi), mais il est possible de recevoir des jours supplémentaires selon certains critères tels que la prise de congés hors période principale ou le nombre de jours travaillés. Par ailleurs, vous pouvez également bénéficier de jours d'absences autorisées à caractère exceptionnel comme un examen pour l'obtention du permis de conduire ou un déménagement."

### Run on questions

In [19]:
import pandas as pd
Q = pd.read_csv("test-for-app-mistral-e5large.csv",sep=";",encoding="latin1")

In [33]:
from tqdm import tqdm
import time

Q["result"] = ""
Q["time"] = ""
Q["context"] = ""
Q["source"] = ""
for i in tqdm(Q.index):
    question = Q["Q"][i]
    start_time = time.time()
    docs = retriever.get_relevant_documents(question, consider_metadata=True)
    response = chain.invoke({"context": docs, "question": question})
    Q["result"][i] = response
    compt = 0
    for j in range(len(docs)):
        Q["context"][i] += str(compt) + " : " + docs[j].page_content +" \n "
        Q["source"][i] += str(compt) + " : " + docs[j].metadata["source"] +" \n "
        compt+=1
    Q["time"][i] = time.time() - start_time
    



 24%|██▍       | 6/25 [1:13:10<3:10:03, 600.19s/it]c:\ProgramData\anaconda3\envs\local-rag\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 28%|██▊       | 7/25 [1:17:55<2:29:03, 496.88s/it]c:\ProgramData\anaconda3\envs\local-rag\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 32%|███▏      | 8/25 [1:22:30<2:00:46, 426.26s/it]c:\ProgramData\anaconda3\envs\local-rag\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `

In [34]:
Q.to_csv("test-for-app-mistral-e5large-result-2.csv",sep=";",encoding="utf-8",index=False)